In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
from dark_emulator_public import dark_emulator
import os, sys, time, json
import matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.interpolate import InterpolatedUnivariateSpline as ius
from scipy.interpolate import interp2d, interp1d
from scipy.integrate import simps
from tqdm import tqdm
import hsc3x2pt

using dark_emulator at  /lustre/work/sunao.sugiyama/package/dark_emulator_public/dark_emulator/__init__.py


## Preparation for Fisher
Computation of $C(l)$ takes ~20 sec for each cosmology.
So this notebook pre-compute $C(l)$ for cosmologies needed for Fisher analysis.

Model parameters of 3x2pt analysis with single(double) source redshift bin are
- comsology: 5
- galaxy bias: 1x3
- magnification bias: 1x3
- photo-z: 1x1(2)
- multiplicative bais: 1

14 parameters in total.

$C(l)$s depend on cosmological parameters, galaxy bias and magnification bias, for which we need to compute $C(l)$.

**Note** : Fisher analysis is doable even if the computational time for one model reaches ~20 sec. However, in the real analysis, ~20 sec is too much to run sampling code. 
I just want to know how people in cosmic shear, like Hikage-san and Hamana-san, or people in 3x2pt, like DES or KiDS, implement code of $C(l)$.?

In [5]:
power_b1 = hsc3x2pt.power_b1_class()

initialize cosmo_class
Initialize pklin emulator
initialize propagator emulator
Initialize sigma_d emulator
initialize cross-correlation emulator
initialize auto-correlation emulator
Initialize sigmaM emulator
initialize xinl emulator


In [6]:
Omega_s_HSC = 140

### List of models to compute $C(l)$ to compute **double** source bins anlaysis

0. fiducial = Planck cosmology + fiducial galaxy bias, magnification bais + 0 photo-z and multiplicative bias
1. $\omega_b h^2$ +
2. $\omega_c h^2$ +
3. $\Omega_{\rm de}$ +
4. $n_{\rm s}$ +
5. $\ln 10^{10}A_{\rm s}$ +
6. $b_{\rm 1,1}$ + 
7. $b_{\rm 1,2}$ + 
8. $b_{\rm 1,3}$ + 
9. $\alpha_{\rm mag,1}$ + 
10. $\alpha_{\rm mag,2}$ + 
11. $\alpha_{\rm mag,3}$ + 
12. $\Delta z_{\rm ph,1}$ +
12. $\Delta z_{\rm ph,2}$ +
13. $\Delta m_1$

In [7]:
dirname = 'double_source_ClY1'

def compute(params, dir_to_save, compute_lens_cross=False):
    t0 = time.time()
    ombh2, omch2, Ode, sigma8, ns, b11, b12, b13, a1, a2, a3, dzph1, dzph2, dm = params
    g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, b11, '40.36 arcmin^-2', a1])
    g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, b12, '20.06 arcmin^-2', a2])
    g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, b13, '30.12 arcmin^-2', a3])
    g_s1 = hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs3.txt', dzph1, dm, 0.2, '4.37 arcmin^-2'])
    g_s2 = hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs4.txt', dzph2, dm, 0.2, '4.37 arcmin^-2'])
    pk2cl = hsc3x2pt.pk2cl_class(power_b1)
    pk2cl.set_galaxy_sample(g_l1)
    pk2cl.set_galaxy_sample(g_l2)
    pk2cl.set_galaxy_sample(g_l3)
    pk2cl.set_galaxy_sample(g_s1)
    pk2cl.set_galaxy_sample(g_s2)
    # set cosmo
    cosmo_dict = dict(zip(['omega_b', 'omega_c', 'Omega_de', 'ln10p10As', 'n_s', 'w_de'], [ombh2, omch2, Ode, 3.094, ns, -1]))
    pk2cl.set_cosmology_from_dict(cosmo_dict)
    sigma8_temp = pk2cl.pk_class.get_sigma8()
    ln10p10As = 3.094 + 2*np.log(sigma8/sigma8_temp)
    print(f'ln10p10As={ln10p10As}')
    cosmo_dict = dict(zip(['omega_b', 'omega_c', 'Omega_de', 'ln10p10As', 'n_s', 'w_de'], [ombh2, omch2, Ode, ln10p10As, ns, -1]))
    pk2cl.set_cosmology_from_dict(cosmo_dict)
    
    pk2cl.init_pk()
    pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

    l = np.logspace(-2, 5, 1000)
    with hsc3x2pt.Time():
        pk2cl.compute_all_Cl(l, compute_lens_cross=compute_lens_cross)
    pk2cl.dump_Cl_cache(f'{dirname}/{dir_to_save}', overwrite=True)
    t1 = time.time()
    print(f'{t1-t0} sec')

In [8]:
dp = 0.01
file_param = od()

def save_dparam(dparam, dir_to_save):
    fname = os.path.join(dirname, 'file_param.json')
    if os.path.exists(fname):
        file_param = json.load(open(fname, 'r'), object_pairs_hook=od) 
    else:
        file_param = od()
    file_param[dir_to_save] = dparam
    json.dump(file_param, open(fname, 'w'), indent=2)

In [9]:
param_fiducial = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0, 0.0]
compute(param_fiducial, 'fiducial', compute_lens_cross=True)

ln10p10As=3.097214912859019
:30.932897567749023 sec
saving l to double_source_ClY1/fiducial/l.txt
saving lowz,lowz to double_source_ClY1/fiducial/lowz,lowz.txt
saving lowz,cmass1 to double_source_ClY1/fiducial/lowz,cmass1.txt
saving lowz,cmass2 to double_source_ClY1/fiducial/lowz,cmass2.txt
saving cmass1,lowz to double_source_ClY1/fiducial/cmass1,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/fiducial/cmass1,cmass1.txt
saving cmass1,cmass2 to double_source_ClY1/fiducial/cmass1,cmass2.txt
saving cmass2,lowz to double_source_ClY1/fiducial/cmass2,lowz.txt
saving cmass2,cmass1 to double_source_ClY1/fiducial/cmass2,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/fiducial/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/fiducial/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/fiducial/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/fiducial/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/fiducial/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/fiducial/cmass2,s

In [10]:
param = [0.02225*(1+dp), 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0, 0.0]
compute(param, 'omega_b')
save_dparam(0.02225*dp, 'omega_b')

ln10p10As=3.1004231384103087
:21.046163082122803 sec
saving l to double_source_ClY1/omega_b/l.txt
saving lowz,lowz to double_source_ClY1/omega_b/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/omega_b/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/omega_b/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/omega_b/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/omega_b/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/omega_b/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/omega_b/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/omega_b/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/omega_b/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/omega_b/s1,s1.txt
saving s1,s2 to double_source_ClY1/omega_b/s1,s2.txt
saving s2,s1 to double_source_ClY1/omega_b/s2,s1.txt
saving s2,s2 to double_source_ClY1/omega_b/s2,s2.txt
saving cosmo_dict to double_source_ClY1/omega_b/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sam

In [11]:
param = [0.02225, 0.1198*(1+dp), 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0, 0.0]
compute(param, 'omega_c')
save_dparam(0.1198*dp, 'omega_c')

ln10p10As=3.082647998157684
:21.660241842269897 sec
saving l to double_source_ClY1/omega_c/l.txt
saving lowz,lowz to double_source_ClY1/omega_c/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/omega_c/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/omega_c/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/omega_c/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/omega_c/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/omega_c/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/omega_c/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/omega_c/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/omega_c/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/omega_c/s1,s1.txt
saving s1,s2 to double_source_ClY1/omega_c/s1,s2.txt
saving s2,s1 to double_source_ClY1/omega_c/s2,s1.txt
saving s2,s2 to double_source_ClY1/omega_c/s2,s2.txt
saving cosmo_dict to double_source_ClY1/omega_c/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy samp

In [12]:
param = [0.02225, 0.1198, 0.6844*(1+dp), 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0, 0.0]
compute(param, 'Omega_de')
save_dparam(0.6844*dp, 'Omega_de')

ln10p10As=3.092198150550478
:21.529731035232544 sec
saving l to double_source_ClY1/Omega_de/l.txt
saving lowz,lowz to double_source_ClY1/Omega_de/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/Omega_de/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/Omega_de/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/Omega_de/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/Omega_de/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/Omega_de/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/Omega_de/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/Omega_de/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/Omega_de/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/Omega_de/s1,s1.txt
saving s1,s2 to double_source_ClY1/Omega_de/s1,s2.txt
saving s2,s1 to double_source_ClY1/Omega_de/s2,s1.txt
saving s2,s2 to double_source_ClY1/Omega_de/s2,s2.txt
saving cosmo_dict to double_source_ClY1/Omega_de/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
sa

In [13]:
param = [0.02225, 0.1198, 0.6844, 0.831*(1.0+dp), 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0, 0.0]
compute(param, 'sigma8')
save_dparam(0.831*dp, 'sigma8')

ln10p10As=3.117115574565356
:21.30530858039856 sec
saving l to double_source_ClY1/sigma8/l.txt
saving lowz,lowz to double_source_ClY1/sigma8/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/sigma8/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/sigma8/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/sigma8/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/sigma8/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/sigma8/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/sigma8/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/sigma8/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/sigma8/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/sigma8/s1,s1.txt
saving s1,s2 to double_source_ClY1/sigma8/s1,s2.txt
saving s2,s1 to double_source_ClY1/sigma8/s2,s1.txt
saving s2,s2 to double_source_ClY1/sigma8/s2,s2.txt
saving cosmo_dict to double_source_ClY1/sigma8/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to gal

In [14]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645*(1.0+dp), 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0, 0.0]
compute(param, 'ns')
save_dparam(0.9645*dp, 'ns')

ln10p10As=3.0899386610164075
:20.921436071395874 sec
saving l to double_source_ClY1/ns/l.txt
saving lowz,lowz to double_source_ClY1/ns/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/ns/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/ns/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/ns/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/ns/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/ns/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/ns/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/ns/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/ns/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/ns/s1,s1.txt
saving s1,s2 to double_source_ClY1/ns/s1,s2.txt
saving s2,s1 to double_source_ClY1/ns/s2,s1.txt
saving s2,s2 to double_source_ClY1/ns/s2,s2.txt
saving cosmo_dict to double_source_ClY1/ns/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galax

In [15]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78*(1.0+dp), 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0, 0.0]
compute(param, 'b1lowz')
save_dparam(1.78*dp, 'b1lowz')

ln10p10As=3.097214912859019
:20.06422519683838 sec
saving l to double_source_ClY1/b1lowz/l.txt
saving lowz,lowz to double_source_ClY1/b1lowz/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/b1lowz/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/b1lowz/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/b1lowz/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/b1lowz/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/b1lowz/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/b1lowz/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/b1lowz/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/b1lowz/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/b1lowz/s1,s1.txt
saving s1,s2 to double_source_ClY1/b1lowz/s1,s2.txt
saving s2,s1 to double_source_ClY1/b1lowz/s2,s1.txt
saving s2,s2 to double_source_ClY1/b1lowz/s2,s2.txt
saving cosmo_dict to double_source_ClY1/b1lowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to gal

In [16]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10*(1.0+dp), 2.28, 2.259, 3.563, 3.729, 0.0, 0.0, 0.0]
compute(param, 'b1cmass1')
save_dparam(2.10*dp, 'b1cmass1')

ln10p10As=3.097214912859019
:19.880629777908325 sec
saving l to double_source_ClY1/b1cmass1/l.txt
saving lowz,lowz to double_source_ClY1/b1cmass1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/b1cmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/b1cmass1/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/b1cmass1/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/b1cmass1/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/b1cmass1/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/b1cmass1/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/b1cmass1/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/b1cmass1/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/b1cmass1/s1,s1.txt
saving s1,s2 to double_source_ClY1/b1cmass1/s1,s2.txt
saving s2,s1 to double_source_ClY1/b1cmass1/s2,s1.txt
saving s2,s2 to double_source_ClY1/b1cmass1/s2,s2.txt
saving cosmo_dict to double_source_ClY1/b1cmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
sa

In [17]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28*(1.0+dp), 2.259, 3.563, 3.729, 0.0, 0.0, 0.0]
compute(param, 'b1cmass2')
save_dparam(2.28*dp, 'b1cmass2')

ln10p10As=3.097214912859019
:19.995863914489746 sec
saving l to double_source_ClY1/b1cmass2/l.txt
saving lowz,lowz to double_source_ClY1/b1cmass2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/b1cmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/b1cmass2/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/b1cmass2/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/b1cmass2/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/b1cmass2/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/b1cmass2/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/b1cmass2/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/b1cmass2/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/b1cmass2/s1,s1.txt
saving s1,s2 to double_source_ClY1/b1cmass2/s1,s2.txt
saving s2,s1 to double_source_ClY1/b1cmass2/s2,s1.txt
saving s2,s2 to double_source_ClY1/b1cmass2/s2,s2.txt
saving cosmo_dict to double_source_ClY1/b1cmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
sa

In [18]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259*(1.0+dp), 3.563, 3.729, 0.0, 0.0, 0.0]
compute(param, 'alphamaglowz')
save_dparam(2.259*dp, 'alphamaglowz')

ln10p10As=3.097214912859019
:20.026432991027832 sec
saving l to double_source_ClY1/alphamaglowz/l.txt
saving lowz,lowz to double_source_ClY1/alphamaglowz/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/alphamaglowz/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/alphamaglowz/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/alphamaglowz/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/alphamaglowz/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/alphamaglowz/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/alphamaglowz/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/alphamaglowz/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/alphamaglowz/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/alphamaglowz/s1,s1.txt
saving s1,s2 to double_source_ClY1/alphamaglowz/s1,s2.txt
saving s2,s1 to double_source_ClY1/alphamaglowz/s2,s1.txt
saving s2,s2 to double_source_ClY1/alphamaglowz/s2,s2.txt
saving cosmo_dict to double_source_ClY1/alphamaglowz/cosmo_dict.

In [19]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563*(1.0+dp), 3.729, 0.0, 0.0, 0.0]
compute(param, 'alphamagcmass1')
save_dparam(3.563*dp, 'alphamagcmass1')

ln10p10As=3.097214912859019
:19.952516794204712 sec
saving l to double_source_ClY1/alphamagcmass1/l.txt
saving lowz,lowz to double_source_ClY1/alphamagcmass1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/alphamagcmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/alphamagcmass1/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/alphamagcmass1/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/alphamagcmass1/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/alphamagcmass1/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/alphamagcmass1/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/alphamagcmass1/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/alphamagcmass1/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/alphamagcmass1/s1,s1.txt
saving s1,s2 to double_source_ClY1/alphamagcmass1/s1,s2.txt
saving s2,s1 to double_source_ClY1/alphamagcmass1/s2,s1.txt
saving s2,s2 to double_source_ClY1/alphamagcmass1/s2,s2.txt
saving cosmo_dict to double_source_C

In [20]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729*(1.0+dp), 0.0, 0.0, 0.0]
compute(param, 'alphamagcmass2')
save_dparam(3.729*dp, 'alphamagcmass2')

ln10p10As=3.097214912859019
:19.97244167327881 sec
saving l to double_source_ClY1/alphamagcmass2/l.txt
saving lowz,lowz to double_source_ClY1/alphamagcmass2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/alphamagcmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/alphamagcmass2/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/alphamagcmass2/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/alphamagcmass2/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/alphamagcmass2/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/alphamagcmass2/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/alphamagcmass2/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/alphamagcmass2/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/alphamagcmass2/s1,s1.txt
saving s1,s2 to double_source_ClY1/alphamagcmass2/s1,s2.txt
saving s2,s1 to double_source_ClY1/alphamagcmass2/s2,s1.txt
saving s2,s2 to double_source_ClY1/alphamagcmass2/s2,s2.txt
saving cosmo_dict to double_source_Cl

In [21]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, dp, 0.0, 0.0]
compute(param, 'dzph1')
save_dparam(dp, 'dzph1')

ln10p10As=3.097214912859019
:20.838845252990723 sec
saving l to double_source_ClY1/dzph1/l.txt
saving lowz,lowz to double_source_ClY1/dzph1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/dzph1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/dzph1/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/dzph1/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/dzph1/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/dzph1/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/dzph1/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/dzph1/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/dzph1/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/dzph1/s1,s1.txt
saving s1,s2 to double_source_ClY1/dzph1/s1,s2.txt
saving s2,s1 to double_source_ClY1/dzph1/s2,s1.txt
saving s2,s2 to double_source_ClY1/dzph1/s2,s2.txt
saving cosmo_dict to double_source_ClY1/dzph1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cma

In [22]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, dp, 0.0]
compute(param, 'dzph2')
save_dparam(dp, 'dzph2')

ln10p10As=3.097214912859019
:19.969465732574463 sec
saving l to double_source_ClY1/dzph2/l.txt
saving lowz,lowz to double_source_ClY1/dzph2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/dzph2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/dzph2/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/dzph2/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/dzph2/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/dzph2/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/dzph2/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/dzph2/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/dzph2/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/dzph2/s1,s1.txt
saving s1,s2 to double_source_ClY1/dzph2/s1,s2.txt
saving s2,s1 to double_source_ClY1/dzph2/s2,s1.txt
saving s2,s2 to double_source_ClY1/dzph2/s2,s2.txt
saving cosmo_dict to double_source_ClY1/dzph2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cma

In [23]:
param = [0.02225, 0.1198, 0.6844, 0.831, 0.9645, 1.78, 2.10, 2.28, 2.259, 3.563, 3.729, 0.0, 0.0, dp]
compute(param, 'dm')
save_dparam(dp, 'dm')

using dark_emulator at  /lustre/work/sunao.sugiyama/package/dark_emulator_public/dark_emulator/__init__.py
ln10p10As=3.097214912859019
:20.0640869140625 sec
saving l to double_source_ClY1/dm/l.txt
saving lowz,lowz to double_source_ClY1/dm/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY1/dm/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY1/dm/cmass2,cmass2.txt
saving lowz,s1 to double_source_ClY1/dm/lowz,s1.txt
saving lowz,s2 to double_source_ClY1/dm/lowz,s2.txt
saving cmass1,s1 to double_source_ClY1/dm/cmass1,s1.txt
saving cmass1,s2 to double_source_ClY1/dm/cmass1,s2.txt
saving cmass2,s1 to double_source_ClY1/dm/cmass2,s1.txt
saving cmass2,s2 to double_source_ClY1/dm/cmass2,s2.txt
saving s1,s1 to double_source_ClY1/dm/s1,s1.txt
saving s1,s2 to double_source_ClY1/dm/s1,s2.txt
saving s2,s1 to double_source_ClY1/dm/s2,s1.txt
saving s2,s2 to double_source_ClY1/dm/s2,s2.txt
saving cosmo_dict to double_source_ClY1/dm/cosmo_dict.json.
saved galaxy sample lowz to galaxy_samp

In [24]:
np.savetxt(os.path.join(dirname, 'param_fiducial.txt'), np.array(param_fiducial))

using dark_emulator at  /lustre/work/sunao.sugiyama/package/dark_emulator_public/dark_emulator/__init__.py
